# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 38

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,20.851064,98,95.395337,4.7,4.575083,4.677341,4.651358,1.987289
411,Patrício,Wolves,53,38.250000,153,162.392030,4.0,4.245543,3.393722,3.595951,1.844146
93,Pope,Burnley,52,37.777778,170,154.753609,4.5,4.096419,3.571189,3.895557,1.796399
471,Henderson,Sheffield Utd,52,36.363636,160,147.356929,4.4,4.052316,3.112521,3.624695,1.777058
168,Schmeichel,Leicester City,55,38.048780,156,152.862626,4.1,4.017543,3.019538,3.751720,1.713086
235,de Gea,Manchester Utd,53,37.631579,143,147.553696,3.8,3.921007,3.673415,3.815909,1.703177
341,Gazzaniga,Tottenham,41,18.125000,58,69.798115,3.2,3.850930,NaN,NaN,1.901839
131,Guaita,Crystal Palace,49,35.128205,137,133.909044,3.9,3.812009,3.578843,3.370637,1.722090
14,Leno,Arsenal,48,30.000000,114,113.489594,3.8,3.782986,NaN,4.693690,1.726689


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,77,38.181818,210,195.313185,5.5,5.115345,4.363091,4.013521,1.843442
660,Thomas,Leicester City,40,3.000000,15,15.121917,5.0,5.040639,3.742733,3.696093,2.520319
103,Alonso,Chelsea,61,17.857143,100,84.951723,5.6,4.757296,5.177721,4.511486,1.926173
181,Robertson,Liverpool,70,36.200000,181,171.705503,5.0,4.743246,6.169807,5.071302,1.792778
401,Doherty,Wolves,65,36.304348,167,170.358897,4.6,4.692521,4.606086,4.689409,1.840558
105,Azpilicueta,Chelsea,61,36.111111,130,161.478957,3.6,4.471725,4.248251,4.603070,1.810549
183,van Dijk,Liverpool,65,37.872340,178,158.034266,4.7,4.172815,4.550760,4.050955,1.636713
297,Lundstram,Sheffield Utd,46,34.285714,144,142.227946,4.2,4.148315,3.642070,3.256647,1.934162
159,Pereira,Leicester City,62,27.954545,123,113.599463,4.4,4.063721,NaN,4.579843,1.632029


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,125,33.768116,233,227.488836,6.9,6.736794,5.629935,6.158719,1.905453
618,Fernandes,Manchester Utd,90,13.928571,117,90.302731,8.4,6.483273,6.147012,6.503317,2.161091
215,De Bruyne,Manchester City,106,34.861111,251,222.634045,7.2,6.386315,7.604783,7.169505,1.961541
214,Sterling,Manchester City,120,32.903226,204,197.049554,6.2,5.988761,7.879749,7.190310,1.728806
192,Mané,Liverpool,121,35.079365,221,195.087692,6.3,5.561323,3.555412,4.121730,1.598768
389,Antonio,West Ham,71,24.130435,111,120.697684,4.6,5.001886,7.264842,7.362975,1.877174
239,Martial,Manchester Utd,85,32.258065,200,160.240448,6.2,4.967454,5.080404,5.639606,1.703823
342,Son,Tottenham,97,30.178571,169,146.807307,5.6,4.864621,4.159195,4.381100,1.561936
431,Pulisic,Chelsea,74,24.901961,127,120.170156,5.1,4.825731,4.515269,5.015948,1.773974


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
166,Vardy,Leicester City,97,35.000000,210,195.078468,6.0,5.573671,5.671272,6.165994,1.789598
233,Rashford,Manchester Utd,89,31.052632,177,170.649678,5.7,5.495498,4.186974,4.748535,1.842095
11,Aubameyang,Arsenal,109,35.964912,205,178.896049,5.7,4.974183,5.606547,5.379025,1.506637
409,Jiménez,Wolves,80,38.039216,194,185.658468,5.1,4.880712,4.060527,4.133609,1.725592
210,Agüero,Manchester City,116,24.000000,132,117.127259,5.5,4.880302,NaN,4.967317,1.432906
313,Ings,Southampton,76,38.076923,198,176.962420,5.2,4.647498,5.603405,5.340992,1.685825
211,Jesus,Manchester City,99,33.953488,146,156.209678,4.3,4.600696,7.347652,6.438208,1.462197
187,Firmino,Liverpool,93,37.804878,155,171.700888,4.1,4.541765,3.509357,3.298347,1.489304
338,Kane,Tottenham,110,29.259259,158,130.862420,5.4,4.472513,6.024738,5.263274,1.348513


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,125,33.768116,233,227.488836,6.9,6.736794,5.629935,6.158719,1.905453
618,Fernandes,Manchester Utd,90,13.928571,117,90.302731,8.4,6.483273,6.147012,6.503317,2.161091
215,De Bruyne,Manchester City,106,34.861111,251,222.634045,7.2,6.386315,7.604783,7.169505,1.961541
214,Sterling,Manchester City,120,32.903226,204,197.049554,6.2,5.988761,7.879749,7.190310,1.728806
166,Vardy,Leicester City,97,35.000000,210,195.078468,6.0,5.573671,5.671272,6.165994,1.789598
192,Mané,Liverpool,121,35.079365,221,195.087692,6.3,5.561323,3.555412,4.121730,1.598768
233,Rashford,Manchester Utd,89,31.052632,177,170.649678,5.7,5.495498,4.186974,4.748535,1.842095
182,Alexander-Arnold,Liverpool,77,38.181818,210,195.313185,5.5,5.115345,4.363091,4.013521,1.843442
660,Thomas,Leicester City,40,3.000000,15,15.121917,5.0,5.040639,3.742733,3.696093,2.520319


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
389,Antonio,West Ham,71,24.130435,111,120.697684,4.6,5.001886,7.264842,7.362975,1.877174
214,Sterling,Manchester City,120,32.903226,204,197.049554,6.2,5.988761,7.879749,7.190310,1.728806
215,De Bruyne,Manchester City,106,34.861111,251,222.634045,7.2,6.386315,7.604783,7.169505,1.961541
618,Fernandes,Manchester Utd,90,13.928571,117,90.302731,8.4,6.483273,6.147012,6.503317,2.161091
211,Jesus,Manchester City,99,33.953488,146,156.209678,4.3,4.600696,7.347652,6.438208,1.462197
166,Vardy,Leicester City,97,35.000000,210,195.078468,6.0,5.573671,5.671272,6.165994,1.789598
191,Salah,Liverpool,125,33.768116,233,227.488836,6.9,6.736794,5.629935,6.158719,1.905453
113,Willian,Chelsea,73,35.744681,168,161.045780,4.7,4.505447,4.331117,6.004263,1.667541
219,David Silva,Manchester City,75,27.254902,139,129.286411,5.1,4.743602,5.752890,5.835858,1.732119


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
660,Thomas,Leicester City,40,3.000000,15,15.121917,5.0,5.040639,3.742733,3.696093,2.520319
618,Fernandes,Manchester Utd,90,13.928571,117,90.302731,8.4,6.483273,6.147012,6.503317,2.161091
340,Lloris,Tottenham,53,20.851064,98,95.395337,4.7,4.575083,4.677341,4.651358,1.987289
215,De Bruyne,Manchester City,106,34.861111,251,222.634045,7.2,6.386315,7.604783,7.169505,1.961541
297,Lundstram,Sheffield Utd,46,34.285714,144,142.227946,4.2,4.148315,3.642070,3.256647,1.934162
103,Alonso,Chelsea,61,17.857143,100,84.951723,5.6,4.757296,5.177721,4.511486,1.926173
191,Salah,Liverpool,125,33.768116,233,227.488836,6.9,6.736794,5.629935,6.158719,1.905453
341,Gazzaniga,Tottenham,41,18.125000,58,69.798115,3.2,3.850930,NaN,NaN,1.901839
389,Antonio,West Ham,71,24.130435,111,120.697684,4.6,5.001886,7.264842,7.362975,1.877174
